In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

import IPython
import IPython.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from load_data import merged_df

import tensorflow as tf
print(tf.__version__)

2.15.0


In [2]:
#load the dataset, slice a smaller amount for now
dataset = merged_df()

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [14]:
REDUCED_DATESET_SIZE = 1_000_000
df = dataset[:REDUCED_DATESET_SIZE].copy()

In [15]:
df

,county,is_business,product_type,target,is_consumption,prediction_datetime,data_block_id,row_id,prediction_unit_id,forecast_date,...,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
2928,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,0.000000,0.043335,0.835190,0.837845,0.971769,-10.245974,0.0,0.0,0.0,0.000090
2929,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,0.000000,0.158173,0.370102,0.393387,0.535612,-5.105104,0.0,0.0,0.0,0.000000
2930,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,0.000000,0.196793,0.158386,0.238663,1.305875,-7.770632,0.0,0.0,0.0,0.000018
2931,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,0.000000,0.119019,0.164139,0.186142,1.239103,-9.215212,0.0,0.0,0.0,0.000080
2932,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,0.000000,0.226669,0.194443,0.290100,1.273527,-9.503542,0.0,0.0,0.0,0.000048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202989,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,0.621887,0.000000,0.000000,0.621887,5.392114,-0.097083,0.0,0.0,0.0,0.000000
1202990,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,0.964600,0.053894,0.000000,0.966278,3.624780,0.146081,0.0,0.0,0.0,0.000000
1202991,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,0.760956,0.347382,0.000000,0.847717,3.048364,0.618249,0.0,0.0,0.0,0.000000
1202992,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,0.846924,0.609528,0.000000,0.942719,3.363793,0.810388,0.0,0.0,0.0,0.000000


In [16]:
#test/train split
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

#drop the target column 
train_labels = train_dataset['target']
test_labels = test_dataset['target']

train_features = train_dataset.drop(columns=['target'])
test_features = test_dataset.drop(columns=['target'])

num_features = len(test_features.columns.values)

In [17]:
#write feature_names to .csv file
import csv
feature_names = list(test_features.columns.values)
with open("../models/dnn_feature_names.csv", 'w') as f:
    write = csv.writer(f)
    write.writerow(feature_names)

In [18]:
#build the three layer model with initial normalization layer
def build_and_compile_model():
  model = keras.Sequential([
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001))
  return model

dnn_model = build_and_compile_model()

#fit the model
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=3)

Epoch 1/3
20000/20000 [==============================] - 104s 5ms/step - loss: 153.6197 - val_loss: 151.9486
Epoch 2/3
20000/20000 [==============================] - 103s 5ms/step - loss: 152.7532 - val_loss: 152.1223
Epoch 3/3
20000/20000 [==============================] - 105s 5ms/step - loss: 153.2432 - val_loss: 152.7661


In [19]:
dnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Bat  (32, 27)                  108       
 chNormalization)                                                
                                                                 
 dense_12 (Dense)            (32, 64)                  1792      
                                                                 
 dense_13 (Dense)            (32, 64)                  4160      
                                                                 
 dense_14 (Dense)            (32, 1)                   65        
                                                                 
Total params: 6125 (23.93 KB)
Trainable params: 6071 (23.71 KB)
Non-trainable params: 54 (216.00 Byte)
_________________________________________________________________


In [21]:
# save the model to disk via pickle
import pickle
dnn_model_filename = '../models/dnn_model.sav'
pickle.dump(dnn_model, open(dnn_model_filename, 'wb'))

In [22]:
#save model to disk via keras
dnn_model.save('../models/dnn_model.keras')

In [23]:
#save model to disk via legacy keras
dnn_model.save('../models/dnn_model.h5')

/Users/jacksonwalters/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
